In [4]:
from tpk4170.visualization import Ur5Visualizer
from dataclasses import dataclass
import modern_robotics as mr
import numpy as np
from kr6 import KR6
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from dh import DHFrame

np.set_printoptions(suppress=True)

PI_HALF = np.pi / 2
PI = np.pi


In [5]:
config1 = np.array([0,0,0,0,0,0])
config2 = np.array([0,-np.pi/2,np.pi/2,0,0,0])
config3 = np.array([0,0,np.pi/2,0,0,0])
config4 = np.array([0,0,0,0,0,0])
config5 = np.array([0,0,0,0,0,0])

kr6_1 = KR6()
print(np.allclose(kr6_1.fk_dh_zero,kr6_1.T_sb_poe,0.01))

kr6 = KR6(config3)
print(kr6.fk_dh_zero)
print(kr6.T_sb_poe)

True
[[ 1.     0.     0.     0.515]
 [ 0.    -1.    -0.    -0.   ]
 [ 0.     0.    -1.    -0.1  ]
 [ 0.     0.     0.     1.   ]]
[[ 1.     0.     0.     0.515]
 [ 0.    -1.     0.     0.   ]
 [ 0.     0.    -1.    -0.1  ]
 [ 0.     0.     0.     1.   ]]


In [6]:
viewer = Viewer(background="white")
viewer.add(Grid())

viewer.add(Axes(0.1))
viewer.add(DHFrame(kr6.Tlist[0]))
viewer.add(DHFrame(kr6.Tlist[1]))
viewer.add(DHFrame(kr6.Tlist[2]))
viewer.add(DHFrame(kr6.Tlist[3]))
viewer.add(DHFrame(kr6.Tlist[4]))
viewer.add(DHFrame(kr6.Tlist[5]))
viewer.add(DHFrame(kr6.Tlist[6]))

C:\Users\Bendik\.conda\envs\tpk4170\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…